In [1]:
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import joblib
import preprocessor as prp

In [2]:
dataset_path = os.path.abspath(r'D:\Program Files (x86)\painting-to-artist\medium_dataset')
training_path = os.path.abspath(r'D:\Program Files (x86)\painting-to-artist\medium_workspace\train')
testing_path = os.path.abspath(os.path.join(r'D:\Program Files (x86)\painting-to-artist\medium_workspace\test'))

prp.clear_files(training_path)
prp.clear_files(testing_path)

n = 50
ratio = 0.8
size = 128
pca_components = 30

x_train, y_train, x_test, y_test, le = prp.preprocess_data(dataset_path, training_path, testing_path, n, ratio, size, pca_components)

In [3]:
RF_model = RandomForestClassifier(n_estimators=50, random_state=0)
RF_model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=0)

In [4]:
test_prediction = RF_model.predict(x_test)

print("RF Accuracy:", metrics.accuracy_score(y_test, test_prediction))

test_prediction = le.inverse_transform(test_prediction)

joblib.dump(RF_model, 'rf_model.pkl')

RF Accuracy: 0.17


['rf_model.pkl']

In [5]:
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [6]:
test_prediction_knn = knn_model.predict(x_test)

print("KNN Accuracy:", metrics.accuracy_score(y_test, test_prediction_knn))

test_prediction_knn = le.inverse_transform(test_prediction_knn)

joblib.dump(RF_model, 'knn_model.pkl')

Accuracy: 0.15


['knn_model.pkl']

In [7]:
svm_model = svm.SVC()
svm_model.fit(x_train, y_train)

SVC()

In [8]:
test_prediction_scv = svm_model.predict(x_test)

print("SVM Accuracy:", metrics.accuracy_score(y_test, test_prediction_scv))

test_prediction_scv = le.inverse_transform(test_prediction_scv)

joblib.dump(RF_model, 'svm_model.pkl')

SVM Accuracy: 0.23


['svm_model.pkl']